In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('all_results/results_direct_code_gen.csv')
# df = pd.read_csv('all_results/results_greedy_extend_pseudocode.csv')
# df = pd.read_csv('all_results/results_step_by_step_code.csv')
# df = pd.read_csv('all_results/results_step_by_step_intermediate.csv')
df

,index,energy,energy_probability,obstacle,start_position,method,actions,model,dataset,prompt_type,movement_dir,carry_limit,cost_per_step,returns_to_start,invalid_move,valid
0,0,5.0,4.000000,block,inner,claude,"['RIGHT', 'DOWNRIGHT', 'DOWNRIGHT', 'TAKE', 'D...",direct_code_gen,inner_cluster_block,claude,eight,100,0.0,True,False,True
1,1,5.0,4.000000,block,inner,claude,"['RIGHT', 'TAKE', 'RIGHT', 'TAKE', 'UPLEFT', '...",direct_code_gen,inner_cluster_block,claude,eight,100,0.0,True,False,True
2,10,7.0,3.000000,block,inner,claude,"['DOWN', 'TAKE', 'LEFT', 'TAKE', 'UP', 'UPRIGH...",direct_code_gen,inner_cluster_block,claude,eight,100,0.0,True,False,True
3,11,7.0,5.000000,block,inner,claude,"['RIGHT', 'RIGHT', 'UPRIGHT', 'TAKE', 'RIGHT',...",direct_code_gen,inner_cluster_block,claude,eight,100,0.0,True,False,True
4,12,7.0,4.000000,block,inner,claude,"['UPLEFT', 'UPLEFT', 'TAKE', 'RIGHT', 'TAKE', ...",direct_code_gen,inner_cluster_block,claude,eight,100,0.0,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136690,95,-5.7,0.365110,free,outer,random,"['LEFT', 'TAKE', 'UP', 'TAKE', 'DOWN', 'TAKE',...",direct_code_gen,outer_upDown_free,random,four,2,0.3,False,True,True
136691,96,-5.7,0.396984,free,outer,random,"['LEFT', 'TAKE', 'RIGHT', 'TAKE', 'RIGHT', 'TA...",direct_code_gen,outer_upDown_free,random,four,2,0.3,True,False,True
136692,97,-5.7,0.612114,free,outer,random,"['UP', 'TAKE', 'RIGHT', 'TAKE', 'UP', 'TAKE', ...",direct_code_gen,outer_upDown_free,random,four,2,0.3,False,True,True
136693,98,-3.7,0.614433,free,outer,random,"['LEFT', 'TAKE', 'UP', 'TAKE', 'UP', 'TAKE', '...",direct_code_gen,outer_upDown_free,random,four,2,0.3,True,False,True


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [4]:
pd.set_option('display.max_rows', None)

In [5]:
df.groupby('prompt_type')['energy'].describe()[['mean', 'std']].round(2)

,mean,std
prompt_type,,
claude,1.30,3.24
deepseek,1.85,2.84
gemini_25_pro,-0.52,2.03
gpt_4o,-1.53,3.61
greedy,0.89,3.29
llama33_70b,-2.38,3.13
o1,0.94,2.78
o3_mini,2.82,3.08
random,-1.77,3.11


In [6]:
def count_actions(actions):
    if actions.replace(' ', '').replace('[', '').replace(']', '') == '':
        return 0
    return min(actions.count(',') + 1, 20)


In [7]:
df['done'] = (df['energy'] + df['cost_per_step'] * df['actions'].apply(count_actions)) > 0
df['done'] = df['done'].astype(int)

In [8]:
df['distribution'] = df['dataset'].apply(lambda x: x.split('_')[1])

In [9]:
# Define your desired order
distribution_order = ['random', 'upDown', 'leftRight', 'cluster', 'spiral']

# Convert 'distribution' to a categorical type with the specified order
df['distribution'] = pd.Categorical(df['distribution'], categories=distribution_order, ordered=True)


In [10]:
df.groupby(['prompt_type', 'obstacle', 'distribution'])['energy'].describe()[['mean', 'std']].round(2)

mean   std
prompt_type   obstacle distribution            
claude        block    random        1.32  3.15
                       upDown        1.19  3.24
                       leftRight     1.15  3.27
                       cluster       1.30  3.31
                       spiral        1.33  3.15
              free     random        1.32  3.17
                       upDown        1.20  3.26
                       leftRight     1.33  3.25
                       cluster       1.51  3.35
                       spiral        1.31  3.18
deepseek      block    random        1.96  2.83
                       upDown        1.87  2.82
                       leftRight     1.85  2.83
                       cluster       1.66  2.93
                       spiral        1.62  2.78
              free     random        2.00  2.83
                       upDown        2.02  2.83
                       leftRight     2.06  2.83
                       cluster       1.73  2.91
                       spiral        1.78  2.77
gemini_25_pro block    random       -0.48  1.85
                       upDown       -0.45  1.97
                       leftRight    -0.50  1.92
                       cluster      -0.80  2.32
                       spiral       -0.54  2.12
              free     random       -0.44  1.81
                       upDown       -0.41  1.83
                       leftRight    -0.38  1.85
                       cluster      -0.79  2.36
                       spiral       -0.46  2.09
gpt_4o        block    random       -1.40  3.65
                       upDown       -1.44  3.61
                       leftRight    -1.46  3.62
                       cluster      -1.93  3.50
                       spiral       -1.61  3.49
              free     random       -1.32  3.68
                       upDown       -1.35  3.71
                       leftRight    -1.26  3.71
                       cluster      -1.96  3.52
                       spiral       -1.56  3.54
greedy        block    random        0.88  3.25
                       upDown        0.88  3.26
                       leftRight     0.88  3.27
                       cluster       0.80  3.29
                       spiral        0.87  3.26
              free     random        0.93  3.32
                       upDown        0.95  3.30
                       leftRight     0.95  3.32
                       cluster       0.82  3.32
                       spiral        0.94  3.30
llama33_70b   block    random       -2.37  3.12
                       upDown       -2.39  3.12
                       leftRight    -2.37  3.13
                       cluster      -2.52  3.09
                       spiral       -2.44  3.12
              free     random       -2.16  3.15
                       upDown       -2.24  3.15
                       leftRight    -2.22  3.15
                       cluster      -2.57  3.11
                       spiral       -2.37  3.14
o1            block    random        1.09  2.77
                       upDown        1.04  2.76
                       leftRight     1.01  2.77
                       cluster       0.50  2.77
                       spiral        0.81  2.72
              free     random        1.16  2.79
                       upDown        1.18  2.78
                       leftRight     1.19  2.79
                       cluster       0.49  2.75
                       spiral        0.90  2.75
o3_mini       block    random        2.89  3.05
                       upDown        2.85  3.05
                       leftRight     2.86  3.07
                       cluster       2.51  3.16
                       spiral        2.60  2.99
              free     random        3.04  3.06
                       upDown        3.03  3.07
                       leftRight     3.10  3.08
                       cluster       2.59  3.20
                       spiral        2.76  3.02
random        block    random       -1.84  3.16
                       upDown       -1.89  3.13
     

In [11]:
df.groupby(['prompt_type', 'cost_per_step', 'distribution'])['energy'].describe()[['mean', 'std']].round(2)

mean   std
prompt_type   cost_per_step distribution            
claude        0.0           random        4.25  1.20
                            upDown        4.14  1.39
                            leftRight     4.18  1.44
                            cluster       4.30  1.71
                            spiral        4.23  1.28
              0.3           random       -1.61  1.15
                            upDown       -1.75  1.36
                            leftRight    -1.70  1.40
                            cluster      -1.48  1.61
                            spiral       -1.59  1.21
deepseek      0.0           random        4.68  0.89
                            upDown        4.65  0.91
                            leftRight     4.66  0.89
                            cluster       4.33  1.34
                            spiral        4.35  0.90
              0.3           random       -0.72  0.75
                            upDown       -0.75  0.77
                            leftRight    -0.76  0.76
                            cluster      -0.95  1.14
                            spiral       -0.96  0.74
gemini_25_pro 0.0           random        1.24  0.67
                            upDown        1.33  0.76
                            leftRight     1.30  0.75
                            cluster       1.33  0.76
                            spiral        1.48  0.73
              0.3           random       -2.16  0.68
                            upDown       -2.19  0.68
                            leftRight    -2.18  0.70
                            cluster      -2.93  1.14
                            spiral       -2.47  0.70
gpt_4o        0.0           random        1.64  2.10
                            upDown        1.59  2.11
                            leftRight     1.63  2.11
                            cluster       1.06  1.82
                            spiral        1.40  1.84
              0.3           random       -4.36  2.11
                            upDown       -4.38  2.13
                            leftRight    -4.35  2.13
                            cluster      -4.94  1.82
                            spiral       -4.58  1.86
greedy        0.0           random        3.68  1.75
                            upDown        3.68  1.76
                            leftRight     3.69  1.77
                            cluster       3.61  1.76
                            spiral        3.67  1.75
              0.3           random       -1.87  1.75
                            upDown       -1.85  1.77
                            leftRight    -1.87  1.76
                            cluster      -1.98  1.76
                            spiral       -1.87  1.75
llama33_70b   0.0           random        0.74  0.92
                            upDown        0.68  0.91
                            leftRight     0.70  0.91
                            cluster       0.35  0.74
                            spiral        0.60  0.89
              0.3           random       -5.26  0.92
                            upDown       -5.32  0.91
                            leftRight    -5.30  0.91
                            cluster      -5.67  0.72
                            spiral       -5.40  0.89
o1            0.0           random        3.84  0.68
                            upDown        3.81  0.68
                            leftRight     3.81  0.70
                            cluster       3.07  1.14
                            spiral        3.52  0.71
              0.3           random       -1.59  0.55
                            upDown       -1.59  0.55
                            leftRight    -1.61  0.57
                            cluster      -2.08  0.81
                            spiral       -1.81  0.55
o3_mini       0.0           random        5.75  1.36
                            upDown        5.73  1.35
                            leftRight     5.78  1.38
                            cluster       5.28  1.79
              